* https://spacy.io 
* https://github.com/astanin/python-tabulate библиотека tabulate для табличного вывода списков
* https://github.com/Textualize/rich библиотека Рич для красивишного вывода в консоль с кучей плюшек (есть гайд на русском)  

Анализ тональности для русскоязычного текста мы будем проводить с помощью библиотеки для анализа настроений Dostoevsky.  
https://egorovegor.ru/analiz-tonalnosti-s-python-i-dostoevsky/ - описание процесса туть  
Для использования библиотеки нужно подгрузить модель FastTextSocialNetworkModel и загрузить RegexTokenizer.

Вкратце о функционале на русском можно читнуть тут 👇.  
Но статья старая, код оттуда лучше не тырить, за 4 года что-то улучшили-переписали-исправили-переделали.  
Актуальные примеры всегда на офф.сайте  
* https://habr.com/ru/articles/531940/
* https://habr.com/ru/articles/504680/

Первый блок 👇 - установка библиотек, выполняется один раз, потом закоментить, чтоб постоянно не проверял, установлено ли или не  
Кстати, для вставки эмодзей можно нажать "Вин + Ю", откроется диалог ✨🎉  



In [1]:
# %pip install spacy
# %pip install tabulate
# %pip install rich         # библиотека, позволяющая отображать красивый текст и форматировать терминал

Тут подключаем:
- всякие спейси-штуки
- Табулейт и Рич приблуды
- создаем Рич-консольку
- создаем объект НЛП для обработки текста

Я закомментировал вывод **стоп-слов** (они хранятся в `nlp.Defaults.stop_words`)  
Не забываем, что дополнительно надо качнуть модель `python -m spacy download ru_core_news_sm` (https://spacy.io/usage/models)  
И примеров (`sentences[0]`) (берем отсюда - `from spacy.lang.ru.examples import sentences`)

In [2]:

import spacy
from spacy.matcher import Matcher
from spacy import displacy
from spacy.lang.ru.examples import sentences
from spacy.lang.ru import Russian

from tabulate import tabulate       # штука для красивого вывода списков в виде таблицы
from rich.console import Console    # Для большего контроля над терминалом
from rich import inspect            # украсить любой Python объект, например класс, переменную, или функцию.
from rich.table import Table        # Рич-таблица

console = Console()     # создаем Рич-консоль

nlp = spacy.load("ru_core_news_md")
# print (nlp.Defaults.stop_words)
# print (sentences[10])

https://spacy.io/usage/visualizers - визуализация текста

In [ ]:
# загоняем в анализатор текст и получаем объект doc
doc = nlp(
    "Машины подобные Буханке скоро сойдут с конвеера. " 
    "Новый автомобиль Лада от фирмы Автоваз поступит в продажу через неделю - заявил генеральный директор Сидоров. Он будет стоить 3 миллиона рублей. "
    "Тыц тырыц")

# так можно посмотреть, а что внутри объекта doc
# inspect(doc, methods=True)  

# вывод текста через Рич-консоль
console.print (":pile_of_poo:")
console.print (doc.text)

# можно вывести несколько слов как срез
# print (doc[0:3])


# print (doc[0].text)

# первообразная форма слова
# print (doc[0].lemma_)

# стоп-слово или не
# print (doc[0].is_stop)

# с помощью дисплейси можно получить картинку с визуализацией текста
# displacy.render(doc, style='dep', jupyter=True)

# serve - показывает через веб-сервер 127.0.0.1:5000
# displacy.serve(doc, style="ent")

Выводим список всех слов с помощью tabulate (делает из списка красивую табличку)

In [ ]:
token_table = []
for token in doc:
    token_table.append ([token.text, token.pos_, token.dep_, token.lemma_])
console.print (tabulate(token_table, headers= ["Текст", "Часть речи", "Чот еще", "Лемма"]))

Тут вывожу список ключевых слов (они хранятся в `doc.ents`) с помощью Ричевской таблицы

In [ ]:
table = Table(header_style="bold red")
table.add_column("Текст")
table.add_column("Label")
table.add_column("Пояснение", justify= "right")

# Iterate over the predicted entities
for ent in doc.ents:

    table.add_row(ent.text, ent.label_, spacy.explain(ent.label_))
    # print(ent.text, ent.label_, spacy.explain(ent.label_))

console.print(table)

Пример, как в тексте найти число `token.like_num` и если за ним следует токен-"ПРОЦЕНТ" - вывести это число.  
У каждого токена есть порядковый номер `token.i`. Соответственно, следующий за ним будет `token.i+1`

In [ ]:
doc1 = nlp ("Буханки составляют 4% от кол-ва машин "
            "В нашем салоне 20% машин - Буханки")
print(doc1[8])

for token in doc1:
    if token.like_num:
        nt = doc1[token.i + 1]
        if nt.text == "%":
            print(token)

https://spacy.io/usage/rule-based-matching - cоздание списка правил  
https://demos.explosion.ai/matcher - можно потестить правила онлайн, но русского там нет 🤷‍♂️

In [ ]:
doc2 = nlp("Привет, мир! привет, мир? привет, мир 111")
#врубаем мэтчер, подключая библиотеку
matcher = Matcher(nlp.vocab)

# Создаем шаблон из 3 токенов
pattern = [{"LOWER": "привет"}, {"IS_PUNCT": True}, {"LOWER": "мир"}]
#pattern = [{"TEXT": "iOS"}, {"IS_DIGIT": True}]
#pattern = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN", "OP": "?"}]

# Добавляем его в мэтчер
matcher.add("HW", [pattern])

# Применяем к тексту
matches = matcher(doc2)
console.print (matches)
console.print("Совпадения:", [doc2[start:end].text for match_id, start, end in matches])

Пример очистки текста от стоп-слов. Это краткая запись, питон позволяет такие фишечки.  
Проходим по токенам в doc1 и если этот токен не стоп-слово - добавляем его лемму в список.  
В блоке ниже - та же операция, но в более понятном виде.

In [9]:
clear_text = []

clear_text = [token.lemma_ for token in doc1 if not token.is_stop]

In [10]:
clear_text = []

for token in doc1:
    if not token.is_stop:
        clear_text.append (token.lemma_)
